<a href="https://colab.research.google.com/github/kvikik/MMO/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 5.2

По итоговому датасету:

1. Преобразовать текстовые данные (очистка + токенизация + нормализация).
2. Обучить модель **Word2Vec** на текстах.
3. Заполнить пропуски в числовых признаках через **SimpleImputer**.
4. Выбрать алгоритм **кластеризации** и построить кластеры.
5. Построить модель для **предсказания кластера** (supervised) с подбором гиперпараметров через **GridSearchCV**.


In [2]:
!pip install pymorphy3 gensim scikit-learn nltk # бибилотеки с которыми будем работать

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 97.2 MB/s eta 0:00:00


In [18]:
# Базовые импорты
import pandas as pd
import numpy as np
import sqlite3

In [21]:
DB_FILE = "articles.sqlite"

conn = sqlite3.connect(DB_FILE)
df = pd.read_sql_query("SELECT * FROM articles;", conn)
conn.close()

In [7]:
df.head()

,guid,title,description,url,published_at,comments_count,created_at_utc,rating
0,fd3a51d1-5cee-4ec2-b53f-effbf1ab74bd,В России вырос МРОТ,Реклама Фото: Александр Сухов / РИА Новости С ...,https://lenta.ru/news/2025/01/01/v-rossii-vyro...,2025-01-01T00:15:00,None,2026-01-18T01:43:00.417204+00:00,None
1,42533e64-a04e-4ce5-8358-b2104dc5f0b2,Миллионы россиян смотрят в Новый год одни и те...,Реклама Фото: pics five / Shutterstock / Fotod...,https://lenta.ru/news/2025/01/01/films/,2025-01-01T00:02:00,None,2026-01-18T01:43:00.548670+00:00,None
2,1d82cea3-8766-4dc4-86b5-008bc9bba49b,Работающим пенсионерам проиндексировали пенсии,Реклама Фото: Elena Mayorova / Globallookpress...,https://lenta.ru/news/2025/01/01/rabotayuschim...,2025-01-01T00:30:00,None,2026-01-18T01:43:00.686638+00:00,None
3,49433220-b1c9-4360-9fb9-c639c4320dcc,В России стало дороже развестись,Реклама Фото: Hutomo Abrianto / Unsplash В Рос...,https://lenta.ru/news/2025/01/01/v-rossii-stal...,2025-01-01T00:02:00,None,2026-01-18T01:43:00.835005+00:00,None
4,7ff041d5-99aa-472b-ac63-be8a74c2afb7,Пенсии россиян проиндексируют,Реклама Фото: Maksim Konstantinov / Global Loo...,https://lenta.ru/news/2025/01/01/pensii-rossiy...,2025-01-01T00:34:00,None,2026-01-18T01:43:00.970848+00:00,None


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9325 entries, 0 to 9324
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   guid            9325 non-null   object
 1   title           9325 non-null   object
 2   description     9325 non-null   object
 3   url             9325 non-null   object
 4   published_at    9325 non-null   object
 5   comments_count  0 non-null      object
 6   created_at_utc  9325 non-null   object
 7   rating          0 non-null      object
dtypes: object(8)
memory usage: 582.9+ KB


## Предобработка

In [23]:
import re
import nltk
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer

In [13]:
#nltk.download("stopwords")

In [24]:
STOP_RU = set(stopwords.words("russian"))
TOKEN_RE = re.compile(r"[А-Яа-яA-Za-z]+", flags=re.U)
morph = MorphAnalyzer()

def preprocess_text(text: str):
    text = str(text).lower()
    tokens = TOKEN_RE.findall(text)
    lemmas = []
    for t in tokens:
        if t in STOP_RU:
            continue
        parsed = morph.parse(t)[0]
        lemmas.append(parsed.normal_form)
    return lemmas

text_col = "description"
tokenized_texts = df[text_col].astype(str).apply(preprocess_text).tolist()


KeyboardInterrupt: 

## Обучение модели

In [20]:
from gensim.models import Word2Vec
import numpy as np

w2v = Word2Vec(
    sentences=tokenized_texts,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=1,
    epochs=10,
)

def doc_vector(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

doc_vectors = np.vstack([doc_vector(t, w2v) for t in tokenized_texts])
doc_vectors.shape


(9325, 100)

## Обработка числовых признаков

In [ ]:
from sklearn.impute import SimpleImputer

num_cols = ["comments_count", "rating"]  # числовые фичи
numeric = df[num_cols].values

imputer = SimpleImputer(strategy="median")
numeric_imputed = imputer.fit_transform(numeric)
numeric_imputed.shape


## Кластеризация


In [ ]:
# В этом блоке подобрать метод кластеризации
# В df записать новую колонку clusfer

In [ ]:
df.cluster = ...

## Предсказать кластеры методом DecisionTree

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

X = X_features
y = df["cluster"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

dt = DecisionTreeClassifier(random_state=42)

param_grid = {
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "criterion": ["gini", "entropy"],
}

grid_dt = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    scoring="accuracy",
    cv=3,
    n_jobs=-1,
)

grid_dt.fit(X_train, y_train)

print("Лучшие параметры:", grid_dt.best_params_)
print("Лучший CV accuracy:", grid_dt.best_score_)

best_dt = grid_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)

print("Test accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))


## Предсказать кластеры методом KNN

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

X = X_features
y = df["cluster"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

knn = KNeighborsClassifier()

param_grid = {
    "n_neighbors": [3, 5, 10, 15],
    "weights": ["uniform", "distance"],
    "metric": ["cosine"],
}

grid = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    scoring="accuracy",
    cv=3,
    n_jobs=-1,
)

grid.fit(X_train, y_train)

print("Лучшие параметры:", grid.best_params_)
print("Лучший CV accuracy:", grid.best_score_)

best_knn = grid.best_estimator_
y_pred = best_knn.predict(X_test)

print("Test accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
